In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

 Originally modified from this example:<br>
 <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/sdk/AI_Platform_(Unified)_SDK_Custom_Training_Python_Package_Managed_Text_Dataset_Tensorflow_Serving_Container.ipynb">Vertex SDK for Python: Custom Training using Python Package, Managed Text Dataset, and TF-Serving Container Example</a>

 If you're looking for an example to build a custom container to serve a scikit-learn model on Vertex Predictions, check this:<br>
 <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/unofficial/sdk/AI_Platform_(Unified)_SDK_Custom_Container_Prediction.ipynb">Vertex SDK for Python: Custom Container Prediction</a>

 Author: Jose Brache <br>
 Email: jbrache@google.com <br>
 <img src="img/google-cloud-icon.jpg" alt="Drawing" style="width: 200px;"/>

# Vertex SDK for Python: Custom Training using Python Package, and scikit-learn Container Example
To use this Jupyter notebook, copy the notebook to a Google Cloud Notebooks instance with Tensorflow installed and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Jupyter automatically displays the return value of the last line in each cell. For more information about running notebooks in Google Cloud Notebook, see the [Google Cloud Notebook guide](https://cloud.google.com/vertex-ai/docs/general/notebooks).

This notebook demonstrate how to create a Custom Model using Custom Python Package Training, and how to serve the model using a pre-built scikit-learn container for online prediction, and batch prediction. It will require you provide a bucket where the dataset will be stored.

**This example uses prebuilt containers for running predictions**, click [here](https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform/notebooks/unofficial/AI_Platform_Custom_Container_Prediction_sklearn.ipynb) for an example on how to build your own container for predictions using scikit-learn.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

### Install Vertex SDK for Python


After the SDK installation the kernel will be automatically restarted. **In general, restart the Kernel once you finish installations**

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Import packages

In [ ]:
# !python3 -m pip install -r ./requirements.txt --user
# !python3 -m pip install kfp==1.6.2
# !python3 -m pip install google-cloud-aiplatform
# !python3 -m pip install google-cloud-storage==1.32
# !python3 -m pip install build
# !gcloud components update --quiet

#### Restart the Kernel once you finished installations!

In [ ]:
# Import packages

import os
import json
import logging
import pandas as pd
import numpy as np
from datetime import datetime
from pytz import timezone
from googleapiclient import discovery
from google.cloud import aiplatform

### Configure Global Variables

List your current GCP project name

In [ ]:
project_id = !gcloud config list --format 'value(core.project)' 2>/dev/null

Configure your system variables

In [ ]:
# Configure your global variables
PROJECT = project_id[0]          # Replace with your project ID
USER = 'test_user'               # Replace with your user name
BUCKET_NAME = project_id[0] + '-vertex-ai'       # Replace with your gcs bucket name

FOLDER_NAME = 'sklearn_models'
ALGORITHM = 'isolation_forest'
TIMEZONE = 'US/Pacific'         
REGION = 'us-central1'           # bucket should be in same region as Vertex AI         
PACKAGE_URIS = f"gs://{BUCKET_NAME}/trainer/{FOLDER_NAME}/{ALGORITHM}/trainer-0.1.tar.gz" 
TRAIN_FEATURE_PATH = f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/train/train.csv"
TEST_FEATURE_PATH = f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/test/test.csv"

In [ ]:
print(f"Project:      {PROJECT}")
print(f"Bucket Name: {BUCKET_NAME}")
print(f"Python Package URI: {PACKAGE_URIS}")
print(f"Training Data URI:  {TRAIN_FEATURE_PATH}")
print(f"Test Data URI:      {TEST_FEATURE_PATH}")

In [ ]:
TASK_TYPE = "custom-py-pkg"
TASK_NAME = f"{TASK_TYPE}"
TASK_DIR = f"./{TASK_NAME}"
DATA_DIR = f"{TASK_DIR}/data"
PYTHON_PACKAGE_APPLICATION_DIR = f"{TASK_NAME}/trainer"

print(f"Task Name:      {TASK_NAME}")
print(f"Task Directory: {TASK_DIR}")
print(f"Data Directory: {DATA_DIR}")
print(f"Python Package Directory: {PYTHON_PACKAGE_APPLICATION_DIR}")

**Create your bucket**

In [ ]:
!gsutil mb -l $REGION gs://$BUCKET_NAME 

### Build python package and upload to your bucket

In [ ]:
!ls $TASK_DIR

In [ ]:
!cd $TASK_DIR && python3 setup.py sdist --formats=gztar

In [ ]:
!ls -ltr $TASK_DIR/dist/trainer-0.1.tar.gz

In [ ]:
!gsutil cp $TASK_DIR/dist/trainer-0.1.tar.gz $PACKAGE_URIS

### Create and upload the dataset

In [ ]:
gcs_source_train_url = TRAIN_FEATURE_PATH
gcs_source_test_url = TEST_FEATURE_PATH
local_source_train = DATA_DIR + "/train/train.csv"
local_source_test = DATA_DIR + "/test/test.csv"

print(f"Train data content will be loaded to {gcs_source_train_url}")
print(f"Local train data content is here {local_source_train}")
print(f"Test data content will be loaded to {gcs_source_train_url}")
print(f"Local test data content is here {local_source_test}")

**Create the dataset**

In [ ]:
rng = np.random.RandomState(42)

# Generate train data
x = 0.3 * rng.randn(100, 2)
x_train = np.r_[x + 2, x - 2]
# Generate some regular novel observations
x = 0.3 * rng.randn(20, 2)
x_test = np.r_[x + 2, x - 2]
# Generate some abnormal novel observations
x_outliers = rng.uniform(low=-4, high=4, size=(20, 2))

if not os.path.exists(DATA_DIR + '/train'):
    os.makedirs(DATA_DIR + '/train')
if not os.path.exists(DATA_DIR + '/test'):
    os.makedirs(DATA_DIR + '/test')
np.savetxt(local_source_train, x_train, fmt='%s', delimiter=",")
np.savetxt(local_source_test, x_test, fmt='%s', delimiter=",")

**Copy the dataset to GCS**

In [ ]:
!gsutil cp $local_source_train $gcs_source_train_url
!gsutil cp $local_source_test $gcs_source_test_url

In [ ]:
path = f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/train"
!gsutil ls $path
path = f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/test"
!gsutil ls $path

------
### Training with Google Vertex AI 

For the full article, please visit: https://cloud.google.com/vertex-ai/docs

Where Vertex AI fits in the ML workflow \
The diagram below gives a high-level overview of the stages in an ML workflow. The blue-filled boxes indicate where Vertex AI provides managed services and APIs:

<img src="img/ml-workflow.svg" alt="Drawing">

As the diagram indicates, you can use Vertex AI to manage the following stages in the ML workflow:

- Train an ML model on your data:
 - Train model
 - Evaluate model accuracy
 - Tune hyperparameters
 
 
- Deploy your trained model.

- Send prediction requests to your model:
 - Online prediction
 - Batch prediction
 
 
- Monitor the predictions on an ongoing basis.

- Manage your models and model versions.


#### Train at local

Before submitting training jobs to Vertex AI, you can test your train.py code in the local environment. You can test by running your python script in command line. This way you can make sure your your entire python package are ready to be submitted to the remote VMs.

In [ ]:
# Get the initial set of hyperparameters
MAX_SAMPLES = '100'  # No of samples
RANDOM_STATE_SEED = '42'

# Train on local machine with python command
!cd $TASK_DIR/trainer && ls
!cd $TASK_DIR && python3 -m trainer.task \
    --input $TRAIN_FEATURE_PATH \
    --job-dir ./models \
    --max-samples $MAX_SAMPLES \
    --random-state-seed $RANDOM_STATE_SEED

In [ ]:
# Print the training feature and label path defined above
print("TRAIN_FEATURE_PATH = ", TRAIN_FEATURE_PATH)

# Vertex AI requires each job to have unique name, 
# Therefore, we use prefix + timestamp to form job names.
JOB_NAME = 'sklearn_isolation_forest_train_{}_{}'.format(
    USER,
    datetime.now(timezone(TIMEZONE)).strftime("%m%d%y_%H%M")
    )
# We use the job names as folder names to store outputs.
JOB_DIR = 'gs://{}/{}/{}'.format(
    BUCKET_NAME,
    FOLDER_NAME,
    JOB_NAME,
    )

print("JOB_NAME_TRN = ", JOB_NAME)
print("JOB_DIR_TRN = ", JOB_DIR)

MAX_SAMPLES = '100'  # No of samples
RANDOM_STATE_SEED = '42'

print("MAX_SAMPLES = ", MAX_SAMPLES)
print("RANDOM_STATE_SEED = ", RANDOM_STATE_SEED)

In [ ]:
executor_image_uri = 'us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-3:latest'
python_module = "trainer.task"
api_endpoint = "us-central1-aiplatform.googleapis.com"
machine_type = "n1-standard-4"
        
# The AI Platform services require regional API endpoints.
client_options = {"api_endpoint": api_endpoint}
# Initialize client that will be used to create and send requests.
# This client only needs to be created once, and can be reused for multiple requests.
client = aiplatform.gapic.JobServiceClient(client_options=client_options)
custom_job = {
    "display_name": JOB_NAME,
    "job_spec": {
        "worker_pool_specs": [
            {
                "machine_spec": {
                    "machine_type": machine_type,
                },
                "replica_count": 1,
                "python_package_spec": {
                    "executor_image_uri": executor_image_uri,
                    "package_uris": [PACKAGE_URIS],
                    "python_module": python_module,
                    "args": [
                      '--input',
                      TRAIN_FEATURE_PATH,
                      '--job-dir',
                      JOB_DIR,
                      '--max-samples',
                      MAX_SAMPLES,
                      '--random-state-seed',
                      RANDOM_STATE_SEED
                    ],
                },
            }
        ]
    },
}
parent = f"projects/{PROJECT}/locations/{REGION}"
response = client.create_custom_job(parent=parent, custom_job=custom_job)
print("response:", response)

**The training job will take about 10-15 minutes to complete.**

Check the training job status

In [ ]:
# check the training job status
job_id_trn = response.name.split('/')[-1]
client_options = {"api_endpoint": api_endpoint}
client = aiplatform.gapic.JobServiceClient(client_options=client_options)
name = client.custom_job_path(
    project=PROJECT,
    location=REGION,
    custom_job=job_id_trn,
)
response = client.get_custom_job(name=name)
print(response.state)

--------
### Deploy the Model

Vertex AI provides tools to upload your trained ML model to the cloud, so that you can send prediction requests to the model.

In order to deploy your trained model on Vertex AI, you must save your trained model using the tools provided by your machine learning framework. This involves serializing the information that represents your trained model into a file which you can deploy for prediction in the cloud.

Then you upload the saved model to a Cloud Storage bucket, and create a model resource on Vertex AI, specifying the Cloud Storage path to your saved model.

When you deploy your model, you can also provide custom code (beta) to customize how it handles prediction requests.



#### Import model artifacts to Vertex AI 

When you import a model, you associate it with a container for Vertex AI to run prediction requests. You can use pre-built containers provided by Vertex AI, or use your own custom containers that you build and push to Container Registry or Artifact Registry.

You can use a pre-built container if your model meets the following requirements:

- Trained in Python 3.7 or later
- Trained using TensorFlow, scikit-learn, or XGBoost
- Exported to meet framework-specific requirements for one of the pre-built prediction containers

The link to the list of pre-built predict container images:

https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers

In [ ]:
MODEL_NAME = "scikit_isolation_forest_model"

response = aiplatform.Model.upload(
    display_name = MODEL_NAME,
    serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-23:latest',
    artifact_uri = JOB_DIR
)

model_id = response.name.split('/')[-1]

#### Create Endpoint

You need the endpoint ID to deploy the model.

In [ ]:
MODEL_ENDPOINT_DISPLAY_NAME = "scikit_isolation_forest_model_endpoint"

aiplatform.init(project=PROJECT, location=REGION)
endpoint = aiplatform.Endpoint.create(
    display_name=MODEL_ENDPOINT_DISPLAY_NAME, project=PROJECT, location=REGION,
)

endpoint_id = endpoint.resource_name.split('/')[-1]

#### Deploy Model to the endpoint

You must deploy a model to an endpoint before that model can be used to serve online predictions; deploying a model associates physical resources with the model so it can serve online predictions with low latency. An undeployed model can serve batch predictions, which do not have the same low latency requirements.

In [ ]:
MODEL_NAME = "scikit_isolation_forest_model"
DEPLOYED_MODEL_DISPLAY_NAME = "scikit_isolation_forest_model_deployed"
aiplatform.init(project=PROJECT, location=REGION)

model = aiplatform.Model(model_name=model_id)

# The explanation_metadata and explanation_parameters should only be
# provided for a custom trained model and not an AutoML model.
model.deploy(
    endpoint=endpoint,
    deployed_model_display_name=DEPLOYED_MODEL_DISPLAY_NAME,
    machine_type = "n1-standard-4",
    sync=True
)

------
### Send inference requests to your model

Vertex AI provides the services you need to request predictions from your model in the cloud.

There are two ways to get predictions from trained models: online prediction (sometimes called HTTP prediction) and batch prediction. In both cases, you pass input data to a cloud-hosted machine-learning model and get inferences for each data instance.

Vertex AI online prediction is a service optimized to run your data through hosted models with as little latency as possible. You send small batches of data to the service and it returns your predictions in the response.

#### Load testing data

#### Call Google API for online inference

In [ ]:
from googleapiclient import errors
import pandas as pd

# Load test feature and labels
x_test = pd.read_csv(TEST_FEATURE_PATH)

# Fill nan value with zeros (Prediction lacks the ability to handle nan values for now)
x_test = x_test.fillna(0)

pprobas = []
batch_size = 10
n_samples = min(160,x_test.shape[0])
print("batch_size=", batch_size)
print("n_samples=", n_samples)

aiplatform.init(project=PROJECT, location=REGION)

for i in range(0, n_samples, batch_size):
    j = min(i+batch_size, n_samples)
    print("Processing samples", i, j)
    response = aiplatform.Endpoint(endpoint_id).predict(instances=x_test.iloc[i:j].values.tolist())
    try:
        for prediction_ in response.predictions:
            pprobas.append(prediction_)
    except errors.HttpError as err:
        # Something went wrong, print out some information.
        tf.compat.v1.logging.error('There was an error getting the job info, Check the details:')
        tf.compat.v1.logging.error(err._get_reason())
        break

In [ ]:
pprobas

#### Call Google GCLOUD API for online inference

In [ ]:
# Load test feature and labels
x_test = pd.read_csv(TEST_FEATURE_PATH)

# Fill nan value with zeros (Prediction lacks the ability to handle nan values for now)
x_test = x_test.fillna(0)

# Create a temporary json file to contain data to be predicted
JSON_TEMP = 'test_data.json' # temp json file name to hold the inference data
batch_size = 100                # data batch size
start = 0
ind = 0
end = min(ind+batch_size, len(x_test))
body={'instances': x_test.iloc[start:end].values.tolist()}
with open(JSON_TEMP, 'w') as fp:
    fp.write(json.dumps(body))

In [ ]:
!gcloud ai endpoints predict $endpoint_id \
  --region=$REGION \
  --json-request=$JSON_TEMP

## Batch Prediction Job on the Model

In [ ]:
aiplatform.init(project=PROJECT, location=REGION)
model_instances = aiplatform.Model.list(
    filter='display_name="scikit_isolation_forest_model"'
)
for resource in model_instances:
    #print(dir(resource))
    print(resource.display_name)
    print(resource.resource_name)
    model_name = resource.resource_name

In [ ]:
print(model_name)
model = aiplatform.Model(model_name=model_name)

In [ ]:
from numpy import genfromtxt

x_test_from_csv = genfromtxt(local_source_test, delimiter=',')
JSON_TEST = 'test_data_batch.json' # temp json file name to hold the inference data
with open(JSON_TEST, 'w') as json_file:
  for row in x_test_from_csv.tolist():
    json.dump(row, json_file)
    json_file.write('\n')

In [ ]:
gcs_source_batch_json_url=f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/batch/input/test_data_batch.json"
gcs_destination_batch_url=f"gs://{BUCKET_NAME}/{FOLDER_NAME}_data/{ALGORITHM}/batch/output"
local_source_batch_json = "test_data_batch.json"

In [ ]:
!gsutil cp $local_source_batch_json $gcs_source_batch_json_url
print(f"Test data content is loaded to {gcs_source_batch_json_url}")

In [ ]:
!gsutil ls $gcs_source_batch_json_url

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=f"temp_{TASK_NAME}_isolation_forest-serving",
    gcs_source=gcs_source_batch_json_url,
    gcs_destination_prefix=gcs_destination_batch_url,
    machine_type="n1-standard-4",
    sync=False,
)

In [ ]:
batch_predict_job.wait()
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_errors_stats = list()
prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.errors_stats"):
        prediction_errors_stats.append(blob.name)
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_results.append(blob.name)

In [ ]:
!gsutil ls $gcs_destination_batch_url

🎉 Congratulations! 🎉

You've learned how to use Vertex AI to:

Train a model by providing the training code in a pre-built container. You used a scikit-learn model in this example, but you can train a model built with any framework using custom containers.
Deploy a scikit-learn model using a pre-built container as part of the same workflow you used for training.
Create a model endpoint and generate a prediction.
Run batch predictions.
To learn more about different parts of Vertex, check out the documentation.
https://cloud.google.com/vertex-ai/docs